# Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Загрузим датасет Wine, разделим на тренировочную и тестовую выборку (random_state=17), используя только [9, 11, 12] признаки.

In [21]:
# библиотеки
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [3]:
# выгружаем датасет
wine_dataset = load_wine()
# разбиваем на X и Y
X = wine_dataset.data
y = wine_dataset.target
df = pd.DataFrame(X, y)

# разбиваем на тренировачный и тестовый датасет
X_train, X_test, y_train, y_test = train_test_split(df.loc[:,[9,11,12]].values, df.index, test_size=0.25, random_state=17)

In [4]:
# задаем тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17
skf = StratifiedKFold(n_splits=5, random_state=17)

### 2.
Обучим логистическую регрессию (LogisticRegression) с параметром *C* по умолчанию и random_state=17. Укажим гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделаем кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведем долю правильных ответов на тестовой выборке.

In [15]:
# задаем логистическую регрессию
log_reg = LogisticRegression(C=1.0, random_state=17, multi_class='ovr', solver='lbfgs')
# обучаем модель
log_reg_model = log_reg.fit(X_train, y_train)
# предсказание
log_reg_pred = log_reg_model.predict(X_test)
# кросс-валидация
log_reg_cross_score = cross_val_score(log_reg_model, X_test, y_test, cv=skf)

In [10]:
print(f'Средняя доля правильных ответов: {np.mean(log_reg_cross_score)}')

Средняя доля правильных ответов: 0.9492063492063492


In [16]:
accuracy = accuracy_score(y_test, log_reg_pred)
print(f'Доля правильных ответов (Accuracy): {accuracy}')

Доля правильных ответов (Accuracy): 0.9111111111111111


### 3.
Обучим метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделаем кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведем среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [19]:
# задаем метод опорных векторов
svc_model = SVC(random_state=17, gamma='scale').fit(X_train, y_train)
# предсказание
svc_pred = svc_model.predict(X_test)
# кросс-валидация
svc_cross_score = cross_val_score(svc_model, X_test, y_test, cv=skf)

In [17]:
print(f'Средняя доля правильных ответов: {np.mean(svc_cross_score)}')

Средняя доля правильных ответов: 0.7847619047619048


In [20]:
svc_accuracy = accuracy_score(y_test, svc_pred)
print(f'Доля правильных ответов (Accuracy): {svc_accuracy}')

Доля правильных ответов (Accuracy): 0.6222222222222222


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии.

### 4.
Для предсказаний обеих моделей построим матрицу ошибок (confusion matrix) и выведем, какие классы каждая из моделей путает больше всего между собой.

In [94]:
# матрица логистической регрессии
conf_mtx_reg = confusion_matrix(y_test, log_reg_pred)
# матрица опорных векторов SVC
conf_mtx_svc = confusion_matrix(y_test, svc_pred)

In [99]:
print(f'LogisticRegression:\n{conf_mtx_reg}\n\nSVC:\n{conf_mtx_svc}')

LogisticRegression:
[[ 9  0  0]
 [ 0 19  0]
 [ 0  4 13]]

SVC:
[[ 9  0  0]
 [ 0 19  0]
 [ 2 15  0]]


LogisticRegression - путает больше всего 3 и 2 класс.<br>
SVC - путает больше всего 1 и 2 класс.

### 5.
Для каждой модели выведем classification report.

In [104]:
print(f'LogisticRegression:\n{classification_report(y_test, log_reg_pred)}\n\nSVC:\n{classification_report(y_test, svc_pred)}')

LogisticRegression:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45


SVC:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45



C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
